In [1]:
distance = 15
doubleplane = 30
energy = 600
neutron = 1
physicss = ["inclxx", "bic", "bert"]

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import joblib
import matplotlib.pyplot as plt
import ROOT
import particle

pd.options.display.min_rows = 100
pd.options.display.max_rows = 100
ROOT.ROOT.EnableThreadSafety()

Welcome to JupyROOT 6.16/00


In [3]:
excluded_particles = [22, 111]

In [4]:
cache = dict()


def pdg_to_name(pdg):
    if pdg in cache:
        return cache[pdg]
    try:
        result = particle.Particle.from_pdgid(pdg).name
    except:
        try:
            npdg = pdg // 10 * 10
            result = particle.Particle.from_pdgid(npdg).name
        except:
            result = str(pdg)
    cache[pdg] = result
    return result

In [5]:
def process(physics):
    simufile = f"output/{distance}m_{doubleplane}dp_{energy}AMeV_{neutron}n_{physics}.simu.root"

    tfile = ROOT.TFile.Open(simufile)
    ttree = tfile.evt

    nEvents = ttree.GetEntries()

    data = []
    for evt in ttree:
        z = -1.0
        secondaries = []
        for nTrack, track in enumerate(evt.MCTrack):
            # Only those created by primary neutrons, no gammas/pi0, no late particles, in the detector
            if (
                (track.GetMotherId() == 0)
                and (track.GetPdgCode() not in excluded_particles)
                and (track.GetStartT() < 1e-5)
                and (track.GetStartZ() > distance * 100)
            ):
                # Convert Be-8 to 2x alpha
                if track.GetPdgCode() == 1000040080:
                    secondaries.append(1000020040)
                    secondaries.append(1000020040)
                else:
                    secondaries.append(track.GetPdgCode())
        secondaries.sort()
        rs = "+".join([pdg_to_name(pdg) for pdg in secondaries])
        data.append(rs)

    tfile.Close()
    del tfile

    counts = Counter(data)
    df = pd.DataFrame([(k, v) for k, v in counts.items()], columns=["Reaction Products", "Counts"])
    df.sort_values(by=["Counts"], ascending=[False], inplace=True)
    df["Frequency [%]"] = df["Counts"] / nEvents * 100
    df = df.reset_index(drop=True)
    return df

In [6]:
for physics in physicss:
    reactions = process(physics)
    print(f"QGSP_{physics.upper()}_HP {reactions.shape[0]}")
    reactions.to_csv(f"results/Reactions-QGSP_{physics.upper()}_HP.csv")
    display(reactions)

QGSP_INCLXX_HP 3279


,Reaction Products,Counts,Frequency [%]
0,n+n+p+D2+He4+He4,7681,7.681
1,n+n+n+p+p+He4+He4,6712,6.712
2,n+p+B11,4900,4.900
3,,4779,4.779
4,n+n+C11,4410,4.410
5,n+n+n+p+p+D2+D2+He4,2670,2.670
6,n+He4+He4+He4,2252,2.252
7,n+p+He4+Li7,2246,2.246
8,n+n+p+He4+Li6,1987,1.987
9,n+n+p+B10,1842,1.842


QGSP_BIC_HP 2579


,Reaction Products,Counts,Frequency [%]
0,n+n+n+p+p+He4+He4,12620,12.620
1,n+n+p+D2+He4+He4,5770,5.770
2,n+p+B11,5321,5.321
3,,4652,4.652
4,n+n+C11,3991,3.991
5,pi-+n+n+p+p+p+He4+He4,2702,2.702
6,n+n+n+n+p+p+p+D2+He4,2386,2.386
7,n+n+p+B10,2235,2.235
8,n+n+p+He4+Li6,2074,2.074
9,n+He4+He4+He4,2043,2.043


QGSP_BERT_HP 2174


,Reaction Products,Counts,Frequency [%]
0,n+p+B11,9781,9.781
1,n+n+n+p+p+He4+He4,9118,9.118
2,n+n+C11,7843,7.843
3,,4790,4.790
4,n+n+p+B10,4111,4.111
5,n+n+p+D2+He4+He4,3995,3.995
6,n+He4+He4+He4,3341,3.341
7,pi-+n+p+C11,2620,2.620
8,pi-+n+n+p+p+p+He4+He4,2135,2.135
9,n+C12,1714,1.714
